# Taxa de rendimento no ensino básico brasileiro
Este notebook será usado para fazer análises baseadas no dataframe disponibilizado pelo governo federal sobre o rendimentos dos alunos de diversas escolas espalhadas pelo país e que pode ser verificado através do link http://dados.gov.br/dataset/taxas-de-rendimento-escolar-na-educacao-basica?fbclid=IwAR1k-zIfTDd7K6rWWhIYY9JPyfkjYI8Pm83tY7_K29ZWntVCx5vMGoPCpMM 

O dataset usado é "Taxas - município 2010"
Serão apresentados os dados recolhidos, as hipóteses criadas, bem como se foram validadas ou não e algumas visualizações e inferências importantes. 

In [ ]:
import pandas as pd
from unicodedata import normalize 

In [ ]:
file = r'data/TX RENDIMENTO MUNICIPIOS 2010.xls'
column_labels = [
    'ano', 'regiao', 'uf', 'codigo_municipio', 'nome_municipio', 'localizacao', 'rede',
    'aprovacao_fundamental_1', 'aprovacao_fundamental_2', 'aprovacao_fundamental_3',
    'aprovacao_fundamental_4', 'aprovacao_fundamental_5', 'aprovacao_fundamental_6',
    'aprovacao_fundamental_7', 'aprovacao_fundamental_8', 'aprovacao_fundamental_9',
    'aprovacao_fundamental_1:5', 'aprovacao_fundamental_6:9', 'aprovacao_fundamental_1:9',
    'aprovacao_medio_1', 'aprovacao_medio_2', 'aprovacao_medio_3', 'aprovacao_medio_4',
    'aprovacao_medio_0', 'aprovacao_medio_0:4',
    'reprovacao_fundamental_1', 'reprovacao_fundamental_2', 'reprovacao_fundamental_3',
    'reprovacao_fundamental_4', 'reprovacao_fundamental_5', 'reprovacao_fundamental_6',
    'reprovacao_fundamental_7', 'reprovacao_fundamental_8', 'reprovacao_fundamental_9',
    'reprovacao_fundamental_1:5', 'reprovacao_fundamental_6:9', 'reprovacao_fundamental_1:9',
    'reprovacao_medio_1', 'reprovacao_medio_2', 'reprovacao_medio_3', 'reprovacao_medio_4',
    'reprovacao_medio_0', 'reprovacao_medio_0:4',
    'abandono_fundamental_1', 'abandono_fundamental_2', 'abandono_fundamental_3',
    'abandono_fundamental_4', 'abandono_fundamental_5', 'abandono_fundamental_6',
    'abandono_fundamental_7', 'abandono_fundamental_8', 'abandono_fundamental_9',
    'abandono_fundamental_1:5', 'abandono_fundamental_6:9', 'abandono_fundamental_1:9',
    'abandono_medio_1', 'abandono_medio_2', 'abandono_medio_3', 'abandono_medio_4',
    'abandono_medio_0', 'abandono_medio_0:4',
]
df_municipios = pd.read_excel(file, skiprows=8, nrows=65460, names=column_labels, na_values='--')

df_municipios.head()

In [ ]:
normalize_string = lambda x: normalize('NFKD', x.upper()).encode('ASCII','ignore').decode("utf-8")

file = r'data/pib_municipios.xls'
df_pib_municipios = pd.read_excel(file, usecols=[0,5], skiprows=8, names=['local', 'pib'], nrows=5601).dropna()
df_pib_municipios['local'] = df_pib_municipios['local'].apply(normalize_string)

df_pib_municipios.head()

In [ ]:
municipio_to_pib = {
    'AROEIRA DO ITAIM': 'AROEIRAS DO ITAIM',
    'BARRA D ALCANTARA': 'BARRA D\'ALCANTARA',
    'BATAIPORA': 'BATAYPORA',
    'BELA VISTA DO CAROBA': 'BELA VISTA DA CAROBA',
    'BELEM DE SAO FRANCISCO': 'BELEM DO SAO FRANCISCO',
    'CHIAPETA': 'CHIAPETTA',
    'COUTO DE MAGALHAES': 'COUTO MAGALHAES',
    'CUVERLANDIA': 'CURVELANDIA',
    'GOIO-ERE': 'GOIOERE',
    'GOVERNADOR LOMANTO JUNIOR': 'BARRO PRETO',
    'ITAMARACA': 'ILHA DE ITAMARACA',
    'LAGOA DO ITAENGA': 'LAGOA DE ITAENGA',
    'OLHO D AGUA DO PIAUI': 'OLHO D\'AGUA DO PIAUI',
    'OLHOS-D AGUA': 'OLHOS-D\'AGUA',
    'PARATI': 'PARATY',
    'PASSA VINTE': 'PASSA-VINTE',
    'PICARRAS': 'BALNEARIO PICARRAS',
    'PINDARE MIRIM': 'PINDARE-MIRIM',
    'PINGO D AGUA': 'PINGO-D\'AGUA',
    'SANTANA DO LIVRAMENTO': 'SANT\'ANA DO LIVRAMENTO',
    'SAO DOMINGOS DE POMBAL': 'POMBAL',
    'SAO FELIPE D OESTE': 'SAO FELIPE D\'OESTE',
    'SAO VALERIO DA NATIVIDADE': 'SAO VALERIO',
    'SAO VICENTE DO SERIDO': 'SERIDO',
    'SEM PEIXE': 'SEM-PEIXE',
    'TOCOS DO MOGI': 'TOCOS DO MOJI',
    'TRAJANO DE MORAIS': 'TRAJANO DE MORAES'
}

pib_to_municipio = {
    'MOJI MIRIM': 'MOGI MIRIM',
}

df_municipios.replace(municipio_to_pib, inplace=True)
df_pib_municipios.replace(pib_to_municipio, inplace=True)

df_municipios = df_municipios[df_municipios.nome_municipio != 'CAMPO DE SANTANA']

pass

In [ ]:
df_municipios = pd.merge(df_municipios, df_pib_municipios, left_on='nome_municipio', right_on='local', how='left')
df_municipios.drop(['local'], axis=1, inplace=True)

cols = df_municipios.columns.tolist()
cols = cols[:6] + cols[-1:] + cols[7:-1]
df_municipios = df_municipios[cols]

df_municipios.head()

In [ ]:
mean_func = lambda col: col.mean()
sum_func = lambda col: col.sum()

agg_map = {
    'pib': sum_func,
}
agg_map.update({
    k: mean_func for k in df_municipios.columns.tolist()[7:]
})

df = df_municipios.dropna().groupby('uf').agg(agg_map)
df